In [10]:
import rospy
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

np.random.BitGenerator = np.random.bit_generator.BitGenerator

In [11]:
import jupyros as jr
import rospy
from std_msgs.msg import String
rospy.init_node('jupyter_node')
jr.subscribe('/sometopic', String, lambda msg: print(msg))

ImportError: this version of pandas is incompatible with numpy < 1.18.5
your numpy version is 1.17.4.
Please upgrade numpy to >= 1.18.5 to use this pandas version

In [12]:
from turtlesim.msg import Pose
jr.live_plot("/turtle1/pose/:x:y", Pose, 10000)

NameError: name 'jr' is not defined

In [5]:
from jupyros import ros3d
v = ros3d.Viewer()
v.objects = [ros3d.GridModel()]

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'

In [4]:
### OPTION 1 ####

from jupyros import ros3d
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
g = ros3d.GridModel()
v.objects = [g, laser_view]

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'